In [1]:
import networkx as nx
import freeman as fm

In [2]:
g = fm.load('marvel.gml')

In [3]:
g.label_nodes('name')
g.set_all_nodes(size=15, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.5))

In [4]:
ec = nx.eigenvector_centrality(g, max_iter=10000)

In [5]:
import pandas as pd
data = pd.DataFrame({
    'id' : [i for i in g.nodes],
    'Name': [g.nodes[i]['name'] for i in g.nodes],
    'eigenvector': [i for i in ec.values()]
})

In [6]:
data.head()

,id,Name,eigenvector
0,1009220,Captain America,0.103293
1,1010740,Winter Soldier,0.058600
2,1009471,Nick Fury,0.087742
3,1009552,S.H.I.E.L.D.,0.044311
4,1009228,Sharon Carter,0.035955


In [7]:
char_infos = pd.read_csv('dataset/charcters_stats.csv', index_col=0)
char_infos.head()

,Alignment,Intelligence,Strength,Speed,Durability,Power,Combat,Total
Name,,,,,,,,
3-D Man,good,50,31,43,32,25,52,233
A-Bomb,good,38,100,17,80,17,64,316
Abe Sapien,good,88,14,35,42,35,85,299
Abin Sur,good,50,90,53,64,84,65,406
Abomination,bad,63,80,53,90,55,95,436


In [8]:
semi_df = pd.merge(data, char_infos, how='inner', on=['Name']) 

In [9]:
semi_df

,id,Name,eigenvector,Alignment,Intelligence,Strength,Speed,Durability,Power,Combat,Total
0,1009220,Captain America,0.103293,good,63,19,35,56,46,100,319
1,1010740,Winter Soldier,0.058600,good,56,32,35,65,60,84,332
2,1009471,Nick Fury,0.087742,good,75,11,23,42,25,100,276
3,1009515,Punisher,0.067364,good,50,16,23,28,22,100,239
4,1009535,Red Skull,0.062462,bad,75,10,12,14,19,80,210
...,...,...,...,...,...,...,...,...,...,...,...
192,1011092,Leech,0.015181,good,25,5,12,14,62,14,132
193,1011071,Demogoblin,0.003091,bad,50,48,42,35,54,60,289
194,1009248,Fabian Cortez,0.007353,bad,1,1,1,1,0,1,5
195,1011073,Bloodaxe,0.003147,bad,63,80,33,80,84,84,424


In [10]:
# Por enquanto não estamos usando os poderes
# powers = pd.read_csv('dataset/superheroes_power_matrix.csv')
# complete_df = pd.merge(semi_df, powers, how='inner', on=['Name'])
# complete_df

In [11]:
gender_values = semi_df['Gender'].unique()
for v in gender_values[:-1]:
    semi_df[v] = [1 if i == v else 0 for i in semi_df['Gender']]

In [12]:
#adicionando a raça dos personagens
semi_df.loc[semi_df["Name"] == "Red Skull", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Emma Frost", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Rogue", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Omega Red", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Machine Man", ["Race"]] = "Cyborg"
semi_df.loc[semi_df["Name"] == "Nightcrawler", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Namor", ["Race"]] = "Atlantean"
semi_df.loc[semi_df["Name"] == "Cloak", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Dagger", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Cypher", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Skaar", ["Race"]] = "Human / Radiation"
semi_df.loc[semi_df["Name"] == "Dormammu", ["Race"]] = "Cosmic Entity"
semi_df.loc[semi_df["Name"] == "Mephisto", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Ego", ["Race"]] = "God / Eternal"
semi_df.loc[semi_df["Name"] == "Man-Thing", ["Race"]] = "Human / Altered"
semi_df.loc[semi_df["Name"] == "Clea", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Wonder Man", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Shadow King", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Hope Summers", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Shatterstar", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Leader", ["Race"]] = "Human / Radiation"
semi_df.loc[semi_df["Name"] == "Wolfsbane", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Forge", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Kang", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Multiple Man", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Cecilia Reyes", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Northstar", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Warbird", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Sage", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Songbird", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Hydro-Man", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Genesis", ["Race"]] = "Mutant / Clone"
semi_df.loc[semi_df["Name"] == "Blizzard", ["Race"]] = "Inhuman"
semi_df.loc[semi_df["Name"] == "Blob", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "X-Man", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Cannonball", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Mimic", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Lady Bullseye", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Namora", ["Race"]] = "Atlantean"
semi_df.loc[semi_df["Name"] == "Vindicator", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Snowbird", ["Race"]] = "Demi-God"
semi_df.loc[semi_df["Name"] == "Ares", ["Race"]] = "God / Eternal"
semi_df.loc[semi_df["Name"] == "Moonstone", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Box", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Meteorite", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Annihilus", ["Race"]] = "Alien"
semi_df.loc[semi_df["Name"] == "Man-Wolf", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Tempest", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Crimson Dynamo", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Chameleon", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Namorita", ["Race"]] = "Mutant / Clone"
semi_df.loc[semi_df["Name"] == "John Wraith", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Thundra", ["Race"]] = "Human / Altered"
semi_df.loc[semi_df["Name"] == "Jigsaw", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Firebird", ["Race"]] = "Human / Radiation"
semi_df.loc[semi_df["Name"] == "Firelord", ["Race"]] = "Alien"
semi_df.loc[semi_df["Name"] == "Tinkerer", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Pyro", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Molten Man", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Feral", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Synch", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Meltdown", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Franklin Storm", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Boomer", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Lyja", ["Race"]] = "Alien"
semi_df.loc[semi_df["Name"] == "Corsair", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Morph", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Stacy X", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Silverclaw", ["Race"]] = "Demi-God"
semi_df.loc[semi_df["Name"] == "Beak", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Morlun", ["Race"]] = "Vampire"
semi_df.loc[semi_df["Name"] == "Stardust", ["Race"]] = "Alien"
semi_df.loc[semi_df["Name"] == "Goblin Queen", ["Race"]] = "Mutant / Clone"
semi_df.loc[semi_df["Name"] == "Callisto", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Agent Zero", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Yellow Claw", ["Race"]] = "Human"
semi_df.loc[semi_df["Name"] == "Shriek", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Arclight", ["Race"]] = "Mutant / Clone"
semi_df.loc[semi_df["Name"] == "Leech", ["Race"]] = "Mutant"
semi_df.loc[semi_df["Name"] == "Fabian Cortez", ["Race"]] = "Mutant"

In [13]:
race_values = semi_df['Race'].unique()
for v in race_values[:-1]:
    semi_df[v] = [1 if i == v else 0 for i in semi_df['Race']]

In [14]:
semi_df = semi_df.drop_duplicates(subset=['Name'], keep='first')

In [15]:
semi_df['Alignment'].value_counts()

good       146
bad         61
neutral      7
Name: Alignment, dtype: int64

In [16]:
semi_df = semi_df[semi_df['Alignment'] != "neutral"]

semi_df['Alignment'] = semi_df['Alignment'].replace('good', 1)
semi_df['Alignment'] = semi_df['Alignment'].replace('bad', 0)

In [17]:
import statsmodels.api as sm

# Linear Regression

indep_variables = []
for i in gender_values[:-1]:
    indep_variables.append(i)
for i in race_values[:-1]:
    indep_variables.append(i)
indep_variables.append('eigenvector')

model = sm.OLS(semi_df['Alignment'], semi_df[[i for i in indep_variables]])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              Alignment   R-squared (uncentered):                   0.769
Model:                            OLS   Adj. R-squared (uncentered):              0.735
Method:                 Least Squares   F-statistic:                              22.25
Date:                Wed, 04 Nov 2020   Prob (F-statistic):                    6.77e-44
Time:                        16:42:15   Log-Likelihood:                         -105.73
No. Observations:                 207   AIC:                                      265.5
Df Residuals:                     180   BIC:                                      355.4
Df Model:                          27                                                  
Covariance Type:            nonrobust                                                  
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Male                 -0.2059      0.128     -1.611      0.109      -0.458       0.046
Female                0.0030      0.136      0.022      0.983      -0.265       0.271
Human                 0.5653      0.139      4.074      0.000       0.292       0.839
Mutant                0.6769      0.129      5.240      0.000       0.422       0.932
Cyborg                0.5336      0.266      2.005      0.046       0.008       1.059
Cosmic Entity         0.0509      0.451      0.113      0.910      -0.839       0.940
Alien                 0.7510      0.219      3.425      0.001       0.318       1.184
Human / Radiation     0.6467      0.198      3.269      0.001       0.256       1.037
Atlantean             0.9377      0.331      2.829      0.005       0.284       1.592
Asgardian             0.5944      0.255      2.335      0.021       0.092       1.097
Mutant / Clone        0.5238      0.234      2.242      0.026       0.063       0.985
Eternal              -0.0263      0.452     -0.058      0.954      -0.918       0.865
Human / Altered       0.7884      0.250      3.158      0.002       0.296       1.281
Zen-Whoberian         0.9349      0.453      2.063      0.041       0.041       1.829
God / Eternal         0.7910      0.209      3.792      0.000       0.379       1.203
Kakarantharaian       1.1022      0.450      2.447      0.015       0.213       1.991
Android               0.3858      0.336      1.149      0.252      -0.277       1.049
Animal                1.1420      0.450      2.535      0.012       0.253       2.031
Human / Cosmic     5.601e-17   1.86e-16      0.301      0.764   -3.11e-16    4.23e-16
Inhuman               0.6926      0.234      2.965      0.003       0.232       1.153
Flora Colossus        1.1766      0.451      2.611      0.010       0.287       2.066
Demi-God              0.8754      0.282      3.101      0.002       0.318       1.433
Vampire               0.6605      0.331      1.996      0.047       0.008       1.313
Symbiote              0.6775      0.331      2.047      0.042       0.024       1.331
Luphomoid            -0.1037      0.453     -0.229      0.819      -0.998       0.791
Maiar                 0.0305      0.451      0.068      0.946      -0.859       0.920
Human-Kree            0.8973      0.453      1.980      0.049       0.003       1.792
eigenvector           4.5937      1.158      3.967      0.000       2.309       6.879
==============================================================================
Omnibus:                       23.192   Durbin-Watson:                   2.124
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               24.073
Skew:                          -0.783   Prob(JB):                     5.92e-06
Ku

In [18]:
# Logistic Regression

model = sm.Logit(semi_df['Alignment'], semi_df[[i for i in indep_variables]])
result = model.fit(method='ncg')
result.summary()

Optimization terminated successfully.
         Current function value: 0.484704
         Iterations: 14
         Function evaluations: 15
         Gradient evaluations: 28
         Hessian evaluations: 14


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Alignment   No. Observations:                  207
Model:                          Logit   Df Residuals:                      180
Method:                           MLE   Df Model:                           26
Date:                Wed, 04 Nov 2020   Pseudo R-squ.:                  0.2005
Time:                        16:42:16   Log-Likelihood:                -100.33
converged:                       True   LL-Null:                       -125.50
Covariance Type:            nonrobust   LLR p-value:                  0.002864
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Male                 -1.6287      0.821     -1.985      0.047      -3.237      -0.020
Female               -0.3316      0.891     -0.372      0.710      -2.078       1.415
Human                 0.6693      0.866      0.773      0.439      -1.027       2.366
Mutant                1.2516      0.827      1.513      0.130      -0.370       2.873
Cyborg                0.3000      1.499      0.200      0.841      -2.638       3.238
Cosmic Entity       -10.8638    328.471     -0.033      0.974    -654.655     632.928
Alien                 1.6366      1.416      1.156      0.248      -1.139       4.412
Human / Radiation     1.1167      1.205      0.927      0.354      -1.245       3.478
Atlantean            12.3100    328.455      0.037      0.970    -631.450     656.070
Asgardian             0.6399      1.467      0.436      0.663      -2.235       3.515
Mutant / Clone        0.3474      1.306      0.266      0.790      -2.213       2.907
Eternal             -11.3136    328.472     -0.034      0.973    -655.106     632.479
Human / Altered       1.8220      1.450      1.256      0.209      -1.020       4.664
Zen-Whoberian        11.5581    328.453      0.035      0.972    -632.198     655.314
God / Eternal         1.7482      1.326      1.319      0.187      -0.850       4.347
Kakarantharaian      12.6134    328.454      0.038      0.969    -631.145     656.371
Android              -0.2370      1.704     -0.139      0.889      -3.576       3.102
Animal               12.8450    328.454      0.039      0.969    -630.913     656.603
Human / Cosmic             0   1.19e+10          0      1.000   -2.33e+10    2.33e+10
Inhuman               1.5035      1.474      1.020      0.308      -1.386       4.393
Flora Colossus       13.0468    328.454      0.040      0.968    -630.711     656.805
Demi-God             12.3583    328.783      0.038      0.970    -632.045     656.761
Vampire               1.3644      1.637      0.834      0.404      -1.843       4.572
Symbiote              1.4633      1.637      0.894      0.371      -1.745       4.671
Luphomoid           -11.8447    328.475     -0.036      0.971    -655.643     631.954
Maiar               -10.9825    328.471     -0.033      0.973    -654.774     632.809
Human-Kree           11.3390    328.453      0.035      0.972    -632.417     655.095
eigenvector          26.7765      7.087      3.778      0.000      12.887      40.666
=====================================================================================
"""